In [1]:
#careful
import pandas as pd #nul
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import json
import geopandas as gpd
from tool_function import *
import altair as alt
from altair import expr, datum
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [10]:
#Sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn import svm
from sklearn.inspection import permutation_importance
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.manifold import TSNE
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.metrics import RocCurveDisplay
from sklearn.base import clone
from sklearn import linear_model
import xgboost

from scipy.spatial import cKDTree
from shapely.geometry import Point, LineString
from shapely.geometry import Point


# Data Prep


In [4]:
mutations = pd.read_pickle('../../data_mutation/geometric_mutations.pkl')
print(f"\nInitial shape:\n{mutations.shape}")
mutations = mutation_process(mutations)
mutations = adjustment_bati(mutations, thresh_sbati=9, thresh_valeur=5000)
mutations = niveau_center_connexion(mutations)
mutations = pop_commune_year(mutations)
mutations = density_commune(mutations)
mutations = salary_connexion(mutations)
mutations = inflation_month(mutations, dir='..')
# mutations = inflation_rate_connexion(mutations)
mutations = get_distances(mutations, dir='..', near=1, distance=1, radius=0.009)


Initial shape:
(1271568, 64)

Original shape mutation process (1271568, 64)
Final shape mutation process (1259641, 41)

Shape before adjustment of bati: (1259641, 41)
Shape after adjustment of bati: (630565, 44), that is  50.06% of original observations



/Users/muhieddineugo/opt/miniconda3/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


# Predictions

In [5]:
features_1 = [
       'coddep',  'vefa',
       'nblot', 'l_codinsee', 'nbsection', 'nbpar',
       'nbparmut', 'nbsuf', 'sterr', 'nbvolmut',
       'nblocmut', 'nblocmai', 'nblocapt', 'nblocdep',
       'nblocact', 'nbapt1pp', 'nbapt2pp', 'nbapt3pp', 'nbapt4pp', 'nbapt5pp',
       'nbmai1pp', 'nbmai2pp', 'nbmai3pp', 'nbmai4pp', 'nbmai5pp', 'sbati',
       'sbatmai', 'sbatapt', 'sbatact', 'sapt1pp', 'sapt2pp', 'sapt3pp',
       'sapt4pp', 'sapt5pp', 'smai1pp', 'smai2pp', 'smai3pp', 'smai4pp',
       'smai5pp', 'codtypbien', 'libtypbien',
       'month', 'year', 'day', 'nivcentr',
       'population', 'salary', 'smoyapt', 'latitude',
       'longitude', 'inflation_rate'
]

features_2 = [
       'month', 'year', 'day', 'salary', 'population',
       'coddep', 'latitude', 'longitude',
        'l_codinsee', 'sbati', 'sbatmai',
       'sbatapt', 'nivcentr', 
        'sterr', 'smoyapt',
        'nblocmai', 'nblocapt', 'nblocact',
        'nblocmut', 'sbatact'
]

features_3 = [
       'year', 'month', 'coddep',
       'libnatmut', 'vefa', 'nblot', 'l_codinsee', 'nbpar',
       'nbparmut', 'nbsuf', 'sterr', 'nbvolmut', 'nblocmut', 'nblocmai',
       'nblocapt', 'nblocdep', 'nblocact', 'sbati', 'sbatmai', 'sbatapt',
       'sbatact', 'sapt1pp', 'sapt2pp', 'sapt3pp', 'sapt4pp', 'sapt5pp',
       'smai1pp', 'smai2pp', 'smai3pp', 'smai4pp', 'smai5pp', 'libtypbien',
       'day',
       'smoyapt', 'nivcentr', 'population', 'dens_pop', 'salary',
       'inflation', 'near_distance', 'near_type', 'near_number', 'latitude',
       'longitude'
]

In [6]:
def model_fitting(X_train, y_train, features, scaler_cols, model, scalery):
    
    scalery.fit(y_train)
    y_train = scalery.transform(y_train)

    x_train_scaled = scaler_cols.fit_transform(X_train)
    model.fit(x_train_scaled, y_train.ravel())

    return scaler_cols, model, scalery
    

def test_output(test_set, features, scaler_cols, model, scalery):
    """
    predicting on the test set
    """
    pred = model.predict(scaler_cols.transform(test_set[features]))
    inv_pred = scalery.inverse_transform([pred]).ravel()
    inv_pred = [elt  if elt > 0 else 5000 for elt in inv_pred]
    
    print(f"R²: {r2_score(test_set.valeurfonc, inv_pred)}")
    print(f"RMSE: {np.sqrt(mean_squared_error(test_set.valeurfonc, inv_pred))}")
    print(f"MSE: {mean_squared_error(test_set.valeurfonc, inv_pred)}")
    print(f"MAE: {mean_absolute_error(test_set.valeurfonc, inv_pred)}")

    return inv_pred

def val_prediction(val_set, y_test, features, scaler_cols, model, scalery):
    """
    predicting on the test set
    """
    pred = model.predict(scaler_cols.transform(val_set[features]))
    inv_pred = scalery.inverse_transform([pred]).ravel()
    inv_pred = [elt  if elt > 0 else 5000 for elt in inv_pred]

    print(f"R²: {r2_score(y_test.valeurfonc, inv_pred)}")
    print(f"RMSE: {np.sqrt(mean_squared_error(y_test.valeurfonc, inv_pred))}")
    print(f"MSE: {mean_squared_error(y_test.valeurfonc, inv_pred)}")
    print(f"MAE: {mean_absolute_error(y_test.valeurfonc, inv_pred)}")

    return pred

In [7]:
mutations.coddep = mutations.coddep.astype('int')

In [32]:
features = features_3
sample_df = mutations[features+['valeurfonc']].copy()#sample(n=150000, random_state=42)
sample_df = sample_df[sample_df.isna().sum(axis=1) == 0].copy()
X = sample_df[features].copy()

Y = sample_df[['valeurfonc']].copy()
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

scalery = MinMaxScaler()
ct = make_column_transformer(
      (StandardScaler(),
        make_column_selector(dtype_include=np.number)),  # rating
       (OneHotEncoder(handle_unknown='ignore'),
        make_column_selector(dtype_include=object)))
model = xgboost.XGBRegressor()
# model = make_pipeline(TruncatedSVD(n_components=100), xgboost.XGBRegressor())

scaler_cols, model, scalery = model_fitting(X_train, y_train, features, ct, model, scalery)

In [30]:
#################
#### VALIDATION SET
#################
val_pred = val_prediction(X_test, y_test, features, scaler_cols, model, scalery)

R²: 0.7902309468666301
RMSE: 80951.73036861714
MSE: 6553182649.673289
MAE: 50182.26649878718


In [18]:
test_set = pd.read_csv('../../data_mutation/test_set.csv', sep=';')


In [19]:
test_set = pd.read_csv('../../data_mutation/test_set.csv', sep=';')
test_set = mutation_test_process(test_set)
print(f"\nInitial shape:\n{test_set.shape}")
test_set = niveau_center_connexion(test_set)
test_set = pop_commune_year(test_set)
test_set = density_commune(test_set)
test_set = salary_connexion(test_set)
test_set = inflation_month(test_set, dir='..')
# mutations = inflation_rate_connexion(mutations)
test_set = get_distances(test_set, dir='..', near=1, distance=1, radius=0.009)


Initial shape:
(2767, 42)


/Users/muhieddineugo/opt/miniconda3/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [31]:
test_pred = test_output(test_set, features, scaler_cols, model, scalery)

R²: -4.188283259281724
RMSE: 285041.8898715975
MSE: 81248878981.57195
MAE: 256668.740238859


In [19]:
model

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [ ]:
model.save_model("svr_model.json")